In [0]:
#import packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


[link to the dataset used](https://drive.google.com/file/d/1Yw9X0n5hf6ZyqqdFjl5IaYj5vlRktbPs/view?usp=sharing)

In [0]:
#importing dataset
movies = pd.read_csv("movies.csv")
ratings =pd.read_csv("ratings.csv")
links =pd.read_csv("links.csv")
tags =pd.read_csv("tags.csv")

In [0]:
dataset1 = movies.merge(ratings,on='movieId', how='inner')
dataset= dataset1.merge(tags,on='movieId', how='inner')

In [22]:
dataset.head()

,movieId,title,genres,userId_x,rating,timestamp_x,userId_y,tag,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,336,pixar,1139045764
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,474,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,567,fun,1525286013
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,336,pixar,1139045764
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,474,pixar,1137206825


In [23]:
#Looking for duplicates
bool_series = dataset["title"].duplicated() 
dataset[bool_series]

,movieId,title,genres,userId_x,rating,timestamp_x,userId_y,tag,timestamp_y
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,474,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,567,fun,1525286013
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,336,pixar,1139045764
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,474,pixar,1137206825
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962,567,fun,1525286013
...,...,...,...,...,...,...,...,...,...
233207,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,586,5.0,1529899556,62,Emilia Clarke,1528934560
233208,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,586,5.0,1529899556,62,star wars,1528934552
233210,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,1537098554,184,comedy,1537098587
233211,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,3.5,1537098554,184,gintama,1537098603


In [24]:
dataset.shape

(233213, 9)

In [0]:
#drop duplicates
dataset1 = dataset[dataset.duplicated(['title', 'userId_y'])]
dataset1.head()
dataset.drop_duplicates(subset =('title', 'userId_y'),
                     keep = "first", inplace = True) 

In [26]:
dataset.shape

(1757, 9)

In [27]:
dataset.info

<bound method DataFrame.info of         movieId                           title  ...            tag  timestamp_y
0             1                Toy Story (1995)  ...          pixar   1139045764
1             1                Toy Story (1995)  ...          pixar   1137206825
2             1                Toy Story (1995)  ...            fun   1525286013
645           2                  Jumanji (1995)  ...        fantasy   1528843929
648           2                  Jumanji (1995)  ...           game   1137375552
...         ...                             ...  ...            ...          ...
233148   183611               Game Night (2018)  ...         Comedy   1526244689
233151   184471              Tomb Raider (2018)  ...      adventure   1528024900
233163   187593               Deadpool 2 (2018)  ...    Josh Brolin   1527274096
233199   187595  Solo: A Star Wars Story (2018)  ...  Emilia Clarke   1528934560
233209   193565       Gintama: The Movie (2010)  ...          anime   1537098

In [28]:
dataset.shape

(1757, 9)

In [29]:
dataset.describe

<bound method NDFrame.describe of         movieId                           title  ...            tag  timestamp_y
0             1                Toy Story (1995)  ...          pixar   1139045764
1             1                Toy Story (1995)  ...          pixar   1137206825
2             1                Toy Story (1995)  ...            fun   1525286013
645           2                  Jumanji (1995)  ...        fantasy   1528843929
648           2                  Jumanji (1995)  ...           game   1137375552
...         ...                             ...  ...            ...          ...
233148   183611               Game Night (2018)  ...         Comedy   1526244689
233151   184471              Tomb Raider (2018)  ...      adventure   1528024900
233163   187593               Deadpool 2 (2018)  ...    Josh Brolin   1527274096
233199   187595  Solo: A Star Wars Story (2018)  ...  Emilia Clarke   1528934560
233209   193565       Gintama: The Movie (2010)  ...          anime   15370

In [30]:
dataset['index'] = range(1, len(dataset) + 1)
dataset.set_index('index',inplace=True)
dataset.head(10)
dataset = dataset.reset_index()
dataset.head(10)

,index,movieId,title,genres,userId_x,rating,timestamp_x,userId_y,tag,timestamp_y
0,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,336,pixar,1139045764
1,2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,474,pixar,1137206825
2,3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703,567,fun,1525286013
3,4,2,Jumanji (1995),Adventure|Children|Fantasy,6,4.0,845553522,62,fantasy,1528843929
4,5,2,Jumanji (1995),Adventure|Children|Fantasy,6,4.0,845553522,474,game,1137375552
5,6,3,Grumpier Old Men (1995),Comedy|Romance,1,4.0,964981247,289,moldy,1143424860
6,7,5,Father of the Bride Part II (1995),Comedy,6,5.0,845553938,474,pregnancy,1137373903
7,8,7,Sabrina (1995),Comedy|Romance,6,4.0,845554264,474,remake,1137375642
8,9,11,"American President, The (1995)",Comedy|Drama|Romance,6,4.0,845553489,474,politics,1137374904
9,10,14,Nixon (1995),Drama,84,4.0,857653349,474,politics,1137375623


In [31]:
#missing values
dataset.isna().sum()

index          0
movieId        0
title          0
genres         0
userId_x       0
rating         0
timestamp_x    0
userId_y       0
tag            0
timestamp_y    0
dtype: int64

**FEATURE SELECTION**

In [0]:
features = ['title','genres','rating','tag']

In [33]:
# Column in the dataset that combines all the selected features

dataset['rating'] = dataset['rating'].astype(str)

for feature in features:
  str(feature)
  
def combined_features(row):
  return row['title']+" "+row['genres']+" "+row['tag']+" "+ row['rating']


# Applying the function to all the rows of the dataframe
dataset['combined_features']=dataset.apply(combined_features,axis=1)
dataset['combined_features'].head()

0    Toy Story (1995) Adventure|Animation|Children|...
1    Toy Story (1995) Adventure|Animation|Children|...
2    Toy Story (1995) Adventure|Animation|Children|...
3    Jumanji (1995) Adventure|Children|Fantasy fant...
4    Jumanji (1995) Adventure|Children|Fantasy game...
Name: combined_features, dtype: object

In [34]:
dataset.dtypes

index                 int64
movieId               int64
title                object
genres               object
userId_x              int64
rating               object
timestamp_x           int64
userId_y              int64
tag                  object
timestamp_y           int64
combined_features    object
dtype: object

In [35]:
dataset.columns

Index(['index', 'movieId', 'title', 'genres', 'userId_x', 'rating',
       'timestamp_x', 'userId_y', 'tag', 'timestamp_y', 'combined_features'],
      dtype='object')

In [0]:
#creating a count matrix for the combined_features
cv=CountVectorizer()
count_matrix = cv.fit_transform(dataset["combined_features"])


In [0]:
#computing cosine similarity for say Truman Show, The (1998)
cosine_similar= cosine_similarity(count_matrix)

In [38]:
#number of rows and columns in the cosine_similar
# used 10000 because the RAM I was allocated was not enough to run trhe wholw dataset
cosine_similar.shape

(1757, 1757)

In [0]:
# function to get title from index
def get_movie_title_from_index(index):
  return dataset[dataset.index==index]["title"].values[0]

In [0]:
#getting the index of this movie from its title
def get_index_from_title(title):
  return dataset[dataset.title==title]["index"].values[0]

In [0]:
# get the title of the movie the user likes
user_movie = "Truman Show, The (1998)"

In [0]:
#find this movies index
user_movie_index= get_index_from_title(user_movie)

In [43]:
user_movie_index

510

In [0]:
#enumerate through all similarity scores of user_movie_index "Truman Show, The (1998)"
#make a tuple of movie index and similarity score
#get similar moivies
similar_movies = list(enumerate(cosine_similar[user_movie_index]))

In [0]:
#sort list according to similarity scores in desceding order
similar_movies_sorted=sorted(similar_movies,key=lambda x:x[1],reverse=True)


In [46]:
# create first 5 entries from the sorted list
# print the first 10 movies
i = 0
print("The top 5 similar movies to ",user_movie," are:" )
for movie in similar_movies_sorted:
  print(get_movie_title_from_index(movie[0]))
  i+=1
  if i>5:
    break

The top 5 similar movies to  Truman Show, The (1998)  are:
Wings of the Dove, The (1997)
Remains of the Day, The (1993)
Lord of the Rings: The Return of the King, The (2003)
Lord of the Rings: The Return of the King, The (2003)
Pride of the Yankees, The (1942)
Passion of the Christ, The (2004)
